In [1]:
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import string


In [2]:
url = 'https://en.wikipedia.org/wiki/Minnesota_House_of_Representatives'
r = requests.get(url)
soup = BeautifulSoup(r.text)
#print(soup.prettify())



/projects/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
legis_tbl = soup.find('table', class_ = "wikitable sortable")
print(legis_tbl)

<table class="wikitable sortable">
<tr>
<th>District</th>
<th>Name</th>
<th>Party</th>
<th>Residence</th>
<th>First elected</th>
</tr>
<tr>
<td data-sort-value="1">1A</td>
<td data-sort-value="Fabian, Dan"><a href="/wiki/Dan_Fabian" title="Dan Fabian">Dan Fabian</a></td>
<td style="background:#FFB6B6">Republican</td>
<td><a href="/wiki/Roseau,_Minnesota" title="Roseau, Minnesota">Roseau</a></td>
<td><a href="/wiki/Minnesota_House_of_Representatives_election,_2010" title="Minnesota House of Representatives election, 2010">2010</a></td>
</tr>
<tr>
<td data-sort-value="2">1B</td>
<td data-sort-value="Kiel, Deb"><a href="/wiki/Deb_Kiel" title="Deb Kiel">Deb Kiel</a></td>
<td style="background:#FFB6B6">Republican</td>
<td><a href="/wiki/Crookston,_Minnesota" title="Crookston, Minnesota">Crookston</a></td>
<td><a href="/wiki/Minnesota_House_of_Representatives_election,_2010" title="Minnesota House of Representatives election, 2010">2010</a></td>
</tr>
<tr>
<td data-sort-value="3">2A</td>
<td

In [4]:
#legis_tbl = soup.find("table", { "class" : "wikitable sortable" })

In [5]:
district = []
name = []
party = []
elected = []
elected2 = []


In [6]:
for row in legis_tbl.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) > 0:
        district.append(cells[0].find(text=True))
        name.append(cells[1].find(text=True))
        party.append(cells[2].find(text=True))
        elected.append(cells[4].find(text=True))


In [7]:
legislator_info = pd.DataFrame(district,columns=['District'])
legislator_info['District'] = district
legislator_info['Name'] = name
legislator_info['Party'] = party
legislator_info['Elected'] = elected
legislator_info.head()

,District,Name,Party,Elected
0,1A,Dan Fabian,Republican,2010
1,1B,Deb Kiel,Republican,2010
2,2A,Dave Hancock,Republican,2010
3,2B,Steve Green,Republican,2012
4,3A,Rob Ecklund,DFL,2015


In [8]:
legislator_info.dtypes


District    object
Name        object
Party       object
Elected     object
dtype: object

If they have non-consecutive terms, then the Elected column just shows the first date. We can modify the append in the web scrape for this section to capture the additional text fields, but I will leave this for another time.


In [9]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s


In [10]:
legislator_info['FirstElected'] = legislator_info['Elected'].apply(remove_punctuation)
legislator_info.head()

,District,Name,Party,Elected,FirstElected
0,1A,Dan Fabian,Republican,2010,2010
1,1B,Deb Kiel,Republican,2010,2010
2,2A,Dave Hancock,Republican,2010,2010
3,2B,Steve Green,Republican,2012,2012
4,3A,Rob Ecklund,DFL,2015,2015


In [11]:
legislator_info['YearElected'] = legislator_info['FirstElected'].astype(int)
legislator_info.dtypes

District        object
Name            object
Party           object
Elected         object
FirstElected    object
YearElected      int64
dtype: object

In [12]:
def newly_elected(x):
    if x['YearElected'] == 2014 or x['YearElected'] == 2015:
        return x['Party']
    else:
        return 'Incumbent'

In [13]:
def new_elected_code(x):
    if x['Pickup'] == 'Republican':
        return 0
    elif x['Pickup'] == 'Incumbent':
        return 1
    else:
        return 2

In [14]:
legislator_info['Pickup'] = legislator_info.apply(newly_elected, axis = 1)
legislator_info['Pickup_code'] = legislator_info.apply(new_elected_code, axis = 1)

In [15]:
leg_info_full = legislator_info
legislator_info.dtypes

District        object
Name            object
Party           object
Elected         object
FirstElected    object
YearElected      int64
Pickup          object
Pickup_code      int64
dtype: object

In [16]:
to_keep = ['District', 'Name', 'Party', 'Pickup', 'Pickup_code', 'YearElected']
legislator_info = leg_info_full[to_keep]
legislator_info

,District,Name,Party,Pickup,Pickup_code,YearElected
0,1A,Dan Fabian,Republican,Incumbent,1,2010
1,1B,Deb Kiel,Republican,Incumbent,1,2010
2,2A,Dave Hancock,Republican,Incumbent,1,2010
3,2B,Steve Green,Republican,Incumbent,1,2012
4,3A,Rob Ecklund,DFL,DFL,2,2015
5,3B,Mary Murphy,DFL,Incumbent,1,1976
6,4A,Ben Lien,DFL,Incumbent,1,2012
7,4B,Paul Marquart,DFL,Incumbent,1,2000
8,5A,John Persell,DFL,Incumbent,1,2008
9,5B,Tom Anzelc,DFL,Incumbent,1,2006


In [19]:
legislator_info.to_csv('legislator_info_2016.csv', header = True, index = False)